## Get json data from Mongo to feed into Neo4j


In [14]:
import pymongo
from utils import mongo_utils
import json

creds = {
    'user': 'root',
    'password': 'example',
    'db': 'LUapp',
    'host':'localhost',
    'port': '27017'
}

In [2]:
mongoCon = mongo_utils.Mongo_Connector(creds=creds)


In [3]:
mydb = mongoCon.client[creds['db']]
mycol = mydb["companies"]


In [17]:
myquery =  [
    {"$addFields": {"size_sim_comps": {"$size": "$similar_companies"}}},
    {"$match": {"size_sim_comps": {"$gt": 0}}},
    {"$project": {"name": 1, "similar_companies": 1, "_id": 0}}
]

In [18]:
mydoc = mycol.aggregate(myquery)

In [19]:
similar_companies = []
for x in mydoc:
  similar_companies.append(x)


In [20]:
len(similar_companies)

2331

In [21]:
similar_companies[0]

{'name': "'47",
 'similar_companies': [{'name': 'New Era Cap',
   'link': 'https://www.linkedin.com/company/new-era-cap',
   'industry': 'Apparel & Fashion',
   'location': 'Buffalo, New York'},
  {'name': 'Life is Good',
   'link': 'https://www.linkedin.com/company/life-is-good',
   'industry': 'Apparel & Fashion',
   'location': 'Boston, MA'},
  {'name': 'Mitchell & Ness Nostalgia Co.',
   'link': 'https://www.linkedin.com/company/mitchell-&-ness',
   'industry': 'Apparel & Fashion',
   'location': None},
  {'name': 'Fanatics, Inc.',
   'link': 'https://www.linkedin.com/company/fanatics-inc-',
   'industry': 'Internet',
   'location': 'Jacksonville, FL., .'},
  {'name': 'New Balance',
   'link': 'https://www.linkedin.com/company/new-balance',
   'industry': 'Sporting Goods',
   'location': 'Brighton, MA'},
  {'name': 'Nike',
   'link': 'https://www.linkedin.com/company/nike',
   'industry': 'Sporting Goods',
   'location': 'Beaverton, OR'},
  {'name': 'Converse',
   'link': 'https://

In [22]:
with open('../data/similar_companies.json','w') as f:
    json.dump(similar_companies, f)